In [2]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
from bs4 import BeautifulSoup as bs

In [ ]:
#기존의 크롤링 방ㅇ법 말고 다른 방식을 써보자

id='mugudeli601'
url = "https://www.law.go.kr/DRF/lawSearch.do?OC={}&target=prec&type=XML".format(id)
response = urlopen(url).read()
bs(response)

In [7]:
xtree = ET.fromstring(response) #response에 저장된 string으로부터 tree를 생성한다

In [27]:
#first check what is in it
'''for child in xtree:
    print(':',child.tag,child.attrib)
print(xtree.find('page').text)'''

1


In [16]:
xtree[3].text #totalCnt

'83460'

In [43]:
for node in xtree[5:]:
    print('\n\n',node.attrib)
    tags=[]
    for child in node:
        tags.append(child.tag)
    for t in tags:
        print(node.find(t).text)



 {'id': '1'}
222677
양수금
2021그753
2022.06.21
대법원
None
민사
400101
결정
자
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222677&type=HTML&mobileYn=


 {'id': '2'}
222693
손해배상(기)
2022다204708
2022.06.16
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222693&type=HTML&mobileYn=


 {'id': '3'}
222697
부당이득금
2022다211850
2022.06.16
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222697&type=HTML&mobileYn=


 {'id': '4'}
222701
회사에관한소송
2022다207967
2022.06.16
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222701&type=HTML&mobileYn=


 {'id': '5'}
222689
가처분이의
2019마6625
2022.06.16
대법원
None
민사
400101
결정
자
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222689&type=HTML&mobileYn=


 {'id': '6'}
222691
토지인도
2018다301350
2022.06.16
대법원
None
민사
400101
판결
선고
/DRF/lawService.do?OC=mugudeli601&target=prec&ID=222691&type=HTML&mobileYn=


 {'id': '7'}
222685
재심기각결정에대한재항고
2022모509
2022.06.16
대법원
None
형사
400102
결정
자
/DRF/lawService

In [8]:
totalCnt = int(xtree.find('totalCnt').text)#판례 페이지에 totalCnt라는 요소가 존재하는 것 같다. 
#왜 20개밖에 없나 했는데 원래 한 페이지당 20개라고 한다. 
#페이지마다 loop을 돌아가야 하는 것

In [48]:
#판례 목록 만들기 
cnt=0
page = 1
rows = []
for i in trange(int(totalCnt / 20)): #기다리는 바를 만드는 tqdm의 range 함수인 trange
    try:
        items = xtree[5:]
    except:
        break
        
    for node in items:
        data={}
        tags=[]
        for child in node:
            tags.append(child.tag)
        for t in tags:
            data[t]=node.find(t).text

        rows.append(data)
    cnt += 1
    url = "https://www.law.go.kr/DRF/lawSearch.do?OC=mugudeli601&target=prec&type=XML&page={}".format(page)
    response = urlopen(url).read()
    xtree = ET.fromstring(response)
    page+=1

  0%|          | 2/4173 [00:00<18:50,  3.69it/s]


In [49]:
#rows에 모든 정보가 담겨있음
prec_list=pd.DataFrame(rows,columns=rows[0].keys())
prec_list.head(5)

,판례일련번호,사건명,사건번호,선고일자,법원명,법원종류코드,사건종류명,사건종류코드,판결유형,선고,판례상세링크
0,221949,임금등,2017다292343,2022.05.26,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
1,221985,대여금[지적장애를 가진 사람의 의사능력이 문제된 사건],2019다213344,2022.05.26,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
2,221993,토지인도등·손해배상금,2021다216421,2022.05.26,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
3,221991,구상금,2020다301186,2022.05.26,대법원,None,민사,400101,판결,선고,/DRF/lawService.do?OC=mugudeli601&target=prec&...
4,221953,상소권회복기각결정에대한재항고,2022모439,2022.05.26,대법원,None,형사,400102,결정,자,/DRF/lawService.do?OC=mugudeli601&target=prec&...


In [55]:
prec_list.columns

Index(['판례일련번호', '사건명', '사건번호', '선고일자', '법원명', '법원종류코드', '사건종류명', '사건종류코드',
       '판결유형', '선고', '판례상세링크'],
      dtype='object')

In [64]:
#prec_list를 traverse하며 모든 판례정보를 긁기
import re

contents = ['판시사항', '판결요지', '참조조문', '참조판례', '판례내용']
case_list=pd.DataFrame(columns=contents)

def remove_tag(content):
    cleaned=re.sub('<.*?>', '', content)
    return cleaned

for pid,l in zip(prec_list['판례일련번호'],prec_list['판례상세링크']):
    url="https://www.law.go.kr"+l.replace('HTML','XML')
    response=urlopen(url).read()
    xtree=ET.fromstring(response)
    row={}
    row['판례일련번호']=pid
    for content in contents:
        text=xtree.find(content).text
        if text is None:
            text="-"
        else:
            text=remove_tag(text)
        row[content]=text
    case_list=pd.concat([case_list,pd.DataFrame([row],columns=row.keys())],axis=0)
case_list.reset_index(inplace=True)
        
        

        

In [65]:
case_list.head(5)

,index,판시사항,판결요지,참조조문,참조판례,판례내용,판례일련번호
0,0,[1] 연령을 이유로 한 차별을 금지하고 있는 구 고용상 연령차별금지 및 고령자...,[1] 구 고용상 연령차별금지 및 고령자고용촉진에 관한 법률(2020. 5. 2...,[1] 구 고용상 연령차별금지 및 고령자고용촉진에 관한 법률(2020. 5. 26...,-,"【원고, 피상고인】 원고 (소송대리인 법무법인(유한) 산경 담당변호사 김선종 외 1...",221949
1,0,[1] 의사능력의 의미 및 의사능력 유무는 구체적인 법률행위와 관련하여 개별적으...,[1] 의사능력이란 자기 행위의 의미나 결과를 정상적인 인식력과 예기력을 바탕으...,"[1] 민법 제9조 제1항, 제12조 제1항, 제14조의2 제1항 / [2] 민법...",[1] 대법원 2002. 10. 11. 선고 2001다10113 판결(공2002하...,"【원고, 피상고인】 비엔케이캐피탈 주식회사【피고, 상고인】 피고(법정대리인 한정후견...",221985
2,0,농지에 관한 임대차계약이 강행법규인 농지법 제23조에 위반되어 무효가 되는 경우...,농지에 관한 임대차계약이 강행법규인 농지법 제23조에 위반되어 무효가 되는 경우...,"농지법 제1조, 제3조, 제23조, 민법 제741조\n","대법원 1994. 6. 28. 선고 93다51539 판결(공1994하, 2089),...","【원고(반소피고), 피상고인】 원고(반소피고) (소송대리인 법무법인 그린 담당변호사...",221993
3,0,[1] 학교안전공제중앙회가 학교배상책임공제에 따라 피해자에게 공제금을 지급한 경...,[1] 학교배상책임공제는 학교안전사고 예방 및 보상에 관한 법률(이하 ‘학교안전...,"[1] 학교안전사고 예방 및 보상에 관한 법률 제15조, 제28조, 상법 제664...",-,"【원고, 피상고인】 학교안전공제중앙회 (소송대리인 법무법인 신의 담당변호사 조주영 ...",221991
4,0,[1] 형사피고사건으로 법원에 재판이 계속 중인 사람은 공소제기 당시의 주소지나...,[1] 형사소송법 제345조의 상소권회복청구는 자기 또는 대리인이 책임질 수 없...,"[1] 형사소송법 제345조 / [2] 형사소송법 제63조 제1항, 제276조, 소...",[1] 대법원 2008. 3. 10. 자 2007모795 결정 / [2] 대법원 ...,【피 고 인】 피고인【재항고인】 피고인【원심결정】 울산지법 2022. 2. 16. ...,221953


In [66]:
case_list.to_csv('case_list.csv'index=False)

'index,판시사항,판결요지,참조조문,참조판례,판례내용,판례일련번호\r\n0,"  [1] 연령을 이유로 한 차별을 금지하고 있는 구 고용상 연령차별금지 및 고령자고용촉진에 관한 법률 제4조의4 제1항이 강행규정인지 여부(적극)  [2] 구 고용상 연령차별금지 및 고령자고용촉진에 관한 법률 제4조의4 제1항에서 말하는 ‘합리적인 이유가 없는’ 경우의 의미 / 사업주가 근로자의 정년을 그대로 유지하면서 임금을 정년 전까지 일정 기간 삭감하는 형태의 이른바 ‘임금피크제’를 시행하는 경우, 연령을 이유로 한 차별에 합리적인 이유가 없어 그 조치가 무효인지 판단하는 기준  [3] 甲 연구원이 노동조합과 신인사제도를 시행하기로 합의한 후 기존의 정년 61세를 그대로 유지하면서 55세 이상 정규직 직원들을 대상으로 임금을 삭감하는 내용의 성과연급제를 시행하였는데, 甲 연구원의 근로자였던 乙이 위 성과연급제는 구 고용상 연령차별금지 및 고령자고용촉진에 관한 법률에 위반되어 무효라고 주장하면서 삭감된 임금 등의 지급을 구한 사안에서, 위 성과연급제는 연령을 이유로 임금 분야에서 乙을 차별하는 것으로 차별에 합리적인 이유가 있다고 볼 수 없다고 한 사례\n","  [1] 구 고용상 연령차별금지 및 고령자고용촉진에 관한 법률(2020. 5. 26. 법률 제17326호로 개정되기 전의 것, 이하 ‘구 고령자고용법’이라 한다) 제4조의4, 제4조의6 제1항, 제4조의7 제1항, 제23조의3 제2항, 제24조 제1항의 내용과 고용의 영역에서 나이를 이유로 한 차별을 금지하여 헌법상 평등권을 실질적으로 구현하려는 구 고령자고용법상 차별 금지 조항의 입법 취지를 고려하면, 구 고령자고용법 제4조의4 제1항은 강행규정에 해당한다. 따라서 단체협약, 취업규칙 또는 근로계약에서 이에 반하는 내용을 정한 조항은 무효이다.  [2] 연령을 이유로 한 차별을 금지하고 있는 구 고용상 연령차별금지 및 고령자고용촉진에 관한 법률(2020. 5. 26. 법률 제17326호로 개정되기 전의 것) 제4조의